In [1]:
# Libraries
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import copy 
import pandas as pd
from scipy.sparse import csr_matrix, lil_matrix
from tqdm.notebook import tnrange, tqdm
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV

In [2]:
## Helper functions
# Check if all elements of array are the same
def all_same(a):
    return len(set(a)) == 1

In [3]:
## Functions that generates data
# Graph
def generate_graph_ama(path, data_limiter, N_limiter):
    # Load graph data
    data = pd.read_csv(path, names=['user', 'item', 'rating', 'time'],
                       usecols=[0, 1])
    ## Data cleaning
    data = data[:data_limiter]
    # Get number of reviews
    user_reviews = data.groupby('user').size().reset_index(name='user_reviews') \
        .sort_values(by='user_reviews', ascending=False).head(N_limiter)
    # Join graph and reviews
    data = data.join(user_reviews.set_index('user'), on='user') \
        .sort_values(by=['user_reviews', 'user', 'item']).dropna()
    # Number of users and items
    N = data.user.unique().size
    K = data.item.unique().size
    # Define id's
    data['user_id'] = data.groupby(['user']).ngroup()
    data['item_id'] = data.groupby(['item']).ngroup()
    # Lil matrices of adjacency and weights
    A_lil = lil_matrix((N, K), dtype=np.float32)
    W_lil = lil_matrix((N, K), dtype=np.float32)
    for _, row in data.iterrows():
        A_lil[row['user_id'], row['item_id']] = 1.0
        W_lil[row['user_id'], row['item_id']] = 1.0 / row['user_reviews']
    # Sparse matrix of weights
    A = A_lil.tocsr()
    W = W_lil.tocsr()
    # Number of edges
    W_num = np.array(copy.deepcopy(data.drop_duplicates(['user_id']).sort_values(by='user_id')['user_reviews'])).reshape((N,1))
    return (A, W, W_num)

# Graph
def generate_graph_sim(N, K_infl, K_noninfl, pi_infl, edge_min, edge_max):
    # Total number of diversion units
    K = K_infl + K_noninfl
    # Adjacency matrix
    A = np.zeros((N,K))
    # Graph weights
    W_num = np.zeros((N,1))
    W = np.zeros((N,K))
    # Connections to influential units
    A[:,:K_infl] = np.random.binomial(1, pi_infl, size=(N,K_infl))
    # For each outcome unit 1,...,N:
    for i in range(N):
        # Number of influential diversion units outcome unit i is connected to
        W_infl_num_i = np.sum(A[i,:K_infl])
        # Number of non-influential diversion units outcome unit i is connected to
        W_noninfl_num_i = max(min(np.random.randint(edge_max + 1), K_noninfl), int(edge_min - W_infl_num_i))
        # Number of all diversion units outcome unit i is connected to
        W_num_i = W_infl_num_i + W_noninfl_num_i
        W_num[i] = W_num_i
        # Which ones out of K_infl+1,...,K
        if W_noninfl_num_i > 0:
            auctions_i = K_infl + np.random.choice(K_noninfl, W_noninfl_num_i, replace=False)
            # Adjacency matrix
            A[i,auctions_i] = 1.0
        # Graph weights (all equal)
        W[i,:] = A[i,:] / W_num_i
    return (A, W, W_num)

# Variables
def generate_vars(mu, A, W, W_num, p, sig_xi, sig_epsilon, e_steps):
    # Range of exposure values to consider
    e_range = np.linspace(0, 1, e_steps)
    # Sample sizes
    (N, K) = A.shape
    # Treatment effects for all outcome units
    beta = np.empty((N,1))
    # For each outcome unit 1,...,N:
    for i in range(N):
        beta[i] = mu(1, W_num[i]) - mu(0, W_num[i])
    # Normalize beta's
    #beta /= np.mean(beta)
    while True:
        # Assignment to treatment for divertion units
        Z = np.random.binomial(1, p, size=(K,1))
        # Exposures
        e = W.dot(Z).reshape((N,1))
        if not all_same(e.flatten()):
            break
    # Treated edges
    W_nonzero = np.rint(e * W_num)
    # Generalized propensity score (observed)
    gps = stats.binom.pmf(W_nonzero, W_num, p)
    # Generalized propensity score (at different exposures)
    gps_at = np.zeros((N,e_steps))
    for k in range(e_steps):
        gps_at[:,k] = stats.binom.pmf(np.rint(e_range[k] * W_num), W_num, p).reshape(N)
    # Error terms at the level of diversion units
    xi = W.dot(np.random.normal(0, sig_xi, (K,1))).reshape((N,1))
    # Outcomes (constant + treatment_effect * exposure + noise)
    y = 0 + mu(e, W_num) + xi + np.random.normal(0, sig_epsilon, (N,1))
    return (y, beta, Z, e, gps, gps_at)

In [4]:
## Function that estimates the true ATE
def ate_true(beta):
    return np.mean(beta)

In [5]:
## Function that estimates a naive model
def est_naive(y, e):
    # Matrix of regressors
    X = sm.add_constant(e)
    # Regress y on exposure and constant
    model = sm.OLS(y, X).fit()
    params_hat = model.params.reshape((-1,1))
    eps_hat = y - model.predict().reshape((-1,1))
    return (params_hat, eps_hat)

def ate_naive(y, e):
    return est_naive(y, e)[0][1]

In [6]:
## Function that estimates the correctly specified model
def ate_correct(y, e, W_num, mu):
    # Matrix of regressors
    X = sm.add_constant(mu(e, W_num))
    # Regress y on exposure and constant
    model = sm.OLS(y, X).fit()
    # Estimated parameter
    b = model.params[1]
    N = W_num.size
    e_0 = np.zeros((N,1))
    e_1 = np.ones((N,1))
    return b * np.mean(mu(e_1, W_num) - mu(e_0, W_num))

In [7]:
## Function that estimates a kernell ridge regression
def ate_kernel(y, e, gps, gps_at):
    # Sample size
    N = y.size
    # Cross-validate the model
    param_grid = {"alpha": [1e0, 1e-1, 1e-2, 1e-3]}
    kr = GridSearchCV(KernelRidge(), param_grid=param_grid)
    X = np.column_stack((e, gps))
    kr.fit(X, y)
    # Get the predictions
    X_0 = np.column_stack((np.zeros((N,1)), gps_at[:,0]))
    y_0 = kr.predict(X_0)
    X_1 = np.column_stack((np.ones((N,1)), gps_at[:,-1]))
    y_1 = kr.predict(X_1)
    #print('{} {}'.format(np.mean(y_1), np.mean(y_0)))
    return np.mean(y_1 - y_0)

In [8]:
## Function that estimates a model with "random coefficients"
def est_random(y, W, Z):
    # Matrix of regressors
    X = sm.add_constant(W)
    # Regress y on weights and a constant
    model = sm.OLS(y, X).fit()
    alpha_hat = model.params[0]
    xi_hat = model.params[1:].reshape((-1,1))
    eps_hat = y - model.predict().reshape((-1,1))
    # Estimate the small model for the right side
    X_small = sm.add_constant(Z)
    model_small = sm.OLS(xi_hat, X_small).fit()
    params_small_hat = model_small.params.reshape((-1,1))
    gamma_hat = xi_hat - model_small.predict().reshape((-1,1))
    return (params_small_hat, alpha_hat, gamma_hat, eps_hat)

def ate_random(y, W, Z):
    return est_random(y, W, Z)[0][1]

In [9]:
## Bootstrap CIs
def compute_bootstrap_CI(y, boot_iter_func, est_func, *args, num_boot_iter=1000, alpha=0.05):
    # Check if all elements are the same
    # Bootstrap iterations
    beta_hat_boot_dist_unsorted = np.zeros(num_boot_iter)
    for b in range(num_boot_iter): #tqdm_notebook(range(num_boot_iter), desc='Bootsrap:'):
        beta_hat_boot_dist_unsorted[b] = boot_iter_func(y, est_func, *args)
    beta_hat_boot_dist = np.sort(beta_hat_boot_dist_unsorted)
    # Sample observations
    q_lo = beta_hat_boot_dist[int(np.floor(num_boot_iter * alpha / 2.0))]
    q_hi = beta_hat_boot_dist[min(int(np.ceil(num_boot_iter * (1.0 - alpha / 2.0))), 
                                  num_boot_iter - 1)] # check for out-of-bounds
    return (q_lo, q_hi)

In [10]:
## Naive bootstrap iteration
def bootstrap_naive_iter(y, est_func, e, *args):
    # Sample size
    N = y.size
    while True:
        # Sample
        boot_sample = np.random.choice(N, N, replace=True)
        # Bootstrap data
        y_b = y[boot_sample,0] # Bootstrap outcomes
        e_b = e[boot_sample,0] # Bootstrap exposures
        if not all_same(e_b):
            break
    args_b = (a if callable(a) else a[boot_sample,:] for a in args)
    return est_func(y_b, e_b, *args_b)

In [11]:
## Parametric bootstrap iteration
def bootstrap_parametric_iter(y, est_func, W, p, params_small_hat, alpha_hat, gamma_hat, eps_hat):
    # Sample sizes
    (N, K) = W.shape
    while True:
        # Bootstrap samples
        boot_sample_out = np.random.choice(N, N, replace=True)
        boot_sample_div = np.random.choice(K, K, replace=True)
        # Bootstrap data
        Z_b = np.random.binomial(1, p, size=(K,1)) # Sample assignments
        gamma_hat_b = gamma_hat[boot_sample_div,:] # Random effects
        xi_hat_b = sm.add_constant(Z_b).dot(params_small_hat) + gamma_hat_b # Main regressor
        eps_hat_b = eps_hat[boot_sample_out,:] # Bootstrap error terms
        if not all_same(xi_hat_b.reshape(K)):
            y_b = alpha_hat + W.dot(xi_hat_b) + eps_hat_b # Recompute outcomes
            break
    return est_func(y_b, W, Z_b)

In [12]:
def main_boot(generate_graph_func, mu,
              e_steps=2, p=0.5, sig_xi=0.25, sig_epsilon=0.25, 
              num_sim_iter=1000, num_boot_iter=1000, alpha=0.05, seed=666,
              *args):
    # Set seed
    np.random.seed(seed)
    # Initialize CI variables
    (ci_naive, ci_param) = (np.zeros((num_sim_iter,2)), np.zeros((num_sim_iter,2)))
    # True value of the parameter
    ate_true_val = np.zeros((num_sim_iter,1))
    # Generate graph
    (A, W, W_num) = generate_graph_func(*args)
    # Main loop for simulations
    for i in tnrange(num_sim_iter, desc='Simulations'):
        (y, beta, Z, e, gps, gps_at) = generate_vars(mu, A, W, W_num, p, sig_xi, sig_epsilon, e_steps)
        # True value
        ate_true_val[i,0] = ate_true(beta)
        # Estimates
        (params_small_hat, alpha_hat, gamma_hat, eps_hat) = est_random(y, W, Z)
        # Naive bootstrap
        (q_lo, q_hi) = compute_bootstrap_CI(y, bootstrap_naive_iter, ate_naive, e, num_boot_iter=num_boot_iter, alpha=alpha)
        ci_naive[i,:] = [q_lo, q_hi]
        # Parametric bootstrap
        (q_lo, q_hi) = compute_bootstrap_CI(y, bootstrap_parametric_iter, ate_random, W, p, params_small_hat, alpha_hat, gamma_hat, eps_hat, num_boot_iter=num_boot_iter, alpha=alpha)
        ci_param[i] = [q_lo, q_hi]
    
    # Coverage / width
    (cov_naive, cov_param) = (np.zeros((num_sim_iter,1)), np.zeros((num_sim_iter,1)))
    for i in range(num_sim_iter):
        cov_naive[i,0] = ci_naive[i,0] <= ate_true_val[i] <= ci_naive[i,1]
        cov_param[i,0] = ci_param[i,0] <= ate_true_val[i] <= ci_param[i,1]
    width_naive = ci_naive[:,1] - ci_naive[:,0]
    width_param = ci_param[:,1] - ci_param[:,0]
        
    # Print the results
    print('Naive bootstrap coverage: {}. Parametric bootstrap coverage: {}' \
         .format(np.mean(cov_naive), np.mean(cov_param)))
    print('Naive mean CI width: {}. Parametric mean CI width: {}' \
         .format(np.mean(width_naive), np.mean(width_param)))
    print('Naive variance of CI width: {}. Parametric variance of CI width: {}' \
         .format(np.var(width_naive), np.var(width_param)))

In [13]:
def main_bias(generate_graph_func, mu,
              e_steps=2, p=0.5, sig_xi=0.0, sig_epsilon=0.25, 
              num_sim_iter=1000, num_boot_iter=1000, alpha=0.05, seed=666,
              *args):
    # Set seed
    np.random.seed(seed)
    # Initialize CI variables
    (ci_naive_naive, ci_naive_correct, ci_naive_kernel) = (np.zeros((num_sim_iter,2)), np.zeros((num_sim_iter,2)), np.zeros((num_sim_iter,2)))
    # True value of the parameter
    ate_true_val = np.zeros((num_sim_iter,1))
    # Estimates of the parameter
    (ate_naive_val, ate_correct_val, ate_kernel_val) = (np.zeros((num_sim_iter,1)), np.zeros((num_sim_iter,1)), np.zeros((num_sim_iter,1)))
    # Generate graph
    (A, W, W_num) = generate_graph_func(*args)
    # Main loop for simulations
    for i in tnrange(num_sim_iter, desc='Simulations'):
        (y, beta, Z, e, gps, gps_at) = generate_vars(mu, A, W, W_num, p, sig_xi, sig_epsilon, e_steps)
        # True value
        ate_true_val[i,0] = ate_true(beta)
        # Naive estimate
        ate_naive_val[i,0] = ate_naive(y, e)
        # Correct estimate
        ate_correct_val[i,0] = ate_correct(y, e, W_num, mu)
        # Kernel Ridge estimate
        ate_kernel_val[i,0] = ate_kernel(y, e, gps, gps_at)
        # Estimates
        # Naive bootstrap naive estimate
        (q_lo, q_hi) = compute_bootstrap_CI(y, bootstrap_naive_iter, ate_naive, e, num_boot_iter=num_boot_iter, alpha=alpha)
        ci_naive_naive[i,:] = [q_lo, q_hi]
        # Naive bootstrap correct estimate
        (q_lo, q_hi) = compute_bootstrap_CI(y, bootstrap_naive_iter, ate_correct, e, W_num, mu, num_boot_iter=num_boot_iter, alpha=alpha)
        ci_naive_correct[i,:] = [q_lo, q_hi]
        # Naive bootstrap kernel estimate
        (q_lo, q_hi) = compute_bootstrap_CI(y, bootstrap_naive_iter, ate_kernel, e, gps, gps_at, num_boot_iter=num_boot_iter, alpha=alpha)
        ci_naive_kernel[i,:] = [q_lo, q_hi]
    
    # Coverage / width
    (cov_naive_naive, cov_naive_correct, cov_naive_kernel) = (np.zeros((num_sim_iter,1)), np.zeros((num_sim_iter,1)), np.zeros((num_sim_iter,1)))
    for i in range(num_sim_iter):
        cov_naive_naive[i,0] = ci_naive_naive[i,0] <= ate_true_val[i] <= ci_naive_naive[i,1]
        cov_naive_correct[i,0] = ci_naive_correct[i,0] <= ate_true_val[i] <= ci_naive_correct[i,1]
        cov_naive_kernel[i,0] = ci_naive_kernel[i,0] <= ate_true_val[i] <= ci_naive_kernel[i,1]
    
    print('Naive bias: {}. Correct specification bias: {}. Kernel bias: {}.' \
         .format(np.mean(ate_naive_val - ate_true_val), np.mean(ate_correct_val - ate_true_val), np.mean(ate_kernel_val - ate_true_val)))
    print('Std. naive bias: {}. Std. correct specification bias: {}. Std. kernel bias: {}.' \
         .format(np.std(ate_naive_val - ate_true_val), np.std(ate_correct_val - ate_true_val), np.std(ate_kernel_val - ate_true_val)))
    print('Naive MRSE: {}. Correct specification MRSE: {}. Kernel MRSE: {}.' \
         .format(np.sqrt(np.mean((ate_naive_val - ate_true_val) ** 2)), np.sqrt(np.mean((ate_correct_val - ate_true_val) ** 2)), np.sqrt(np.mean((ate_kernel_val - ate_true_val) ** 2))))
    print('Naive bootstrap naive estimate coverage: {}. Naive bootstrap correct specification estimate coverage: {}. Naive bootstrap kernel estimate coverage: {}.' \
         .format(np.mean(cov_naive_naive), np.mean(cov_naive_correct), np.mean(cov_naive_kernel)))

In [14]:
## Parameters
path, data_limiter, N_limiter = 'item_dedup_truncated.csv', 50_000, 1_000
N, K_infl, K_noninfl, pi_infl, edge_min, edge_max = 1_000, 0, 100, 0.0, 1, 10
sig_xi_uncorrelated = 0.0
sig_xi_correlated = 0.25
sig_epsilon = 0.25
e_steps = 2
p = 0.5
num_sim_iter = 100
num_boot_iter = 200
alpha = 0.05
seed = 666
def mu_homogeneous(e, W_num):
    return e * ((edge_min + edge_max) / 2.0)
def mu_heterogeneous(e, W_num):
    return e * W_num

In [ ]:
## Simulated data, homogeneous treatments
main_bias(# Common parameters
          generate_graph_sim, mu_homogeneous,
          e_steps, p, sig_xi_uncorrelated, sig_epsilon, 
          num_sim_iter, num_boot_iter, alpha, seed,
          # Data parameters
          N, K_infl, K_noninfl, pi_infl, edge_min, edge_max)

In [ ]:
## Simulated data, heterogeneous treatments
main_bias(# Common parameters
          generate_graph_sim, mu_heterogeneous,
          e_steps, p, sig_xi_uncorrelated, sig_epsilon, 
          num_sim_iter, num_boot_iter, alpha, seed,
          # Data parameters
          N, K_infl, K_noninfl, pi_infl, edge_min, edge_max)

In [ ]:
## Amazon graph, homogeneous treatments
main_bias(# Common parameters
          generate_graph_ama, mu_homogeneous,
          e_steps, p, sig_xi_uncorrelated, sig_epsilon, 
          num_sim_iter, num_boot_iter, alpha, seed,
          # Data parameters
          path, data_limiter, N_limiter)

In [ ]:
## Amazon graph, heterogeneous treatments
main_bias(# Common parameters
          generate_graph_ama, mu_heterogeneous,
          e_steps, p, sig_xi_uncorrelated, sig_epsilon, 
          num_sim_iter, num_boot_iter, alpha, seed,
          # Data parameters
          path, data_limiter, N_limiter)

In [ ]:
## Simulated graph, parametric bootstrap for correlated errors
main_boot(# Common parameters
          generate_graph_sim, mu_homogeneous,
          e_steps, p, sig_xi_correlated, sig_epsilon,
          num_sim_iter, num_boot_iter, alpha, seed,
          # Data parameters
          N, K_infl, K_noninfl, pi_infl, edge_min, edge_max)